In [ ]:
import io, os, sys, types
from IPython import get_ipython
from Utils import NotebookFinder
import tensorflow as tf
import numpy as np
import model
import matplotlib.pyplot as plt
import os
import pandas as pd
import Train_model

In [ ]:
n_test = 3900
flag = 0
num_Iteration = 5026
BATCH_SIZE = model.BATCH_SIZE

In [ ]:
def evaluation():
    with tf.Graph().as_default():   
        features = tf.placeholder(tf.float32, [None,81])
        labels = tf.placeholder(tf.int32, [None])
        keep_prob = tf.placeholder(tf.float32)
        global_step = tf.Variable(0, trainable=False)
        t_logits = model.inference(features,Train_model.REGULARIZER,keep_prob)
        reshape_logits = tf.cast(tf.argmax(t_logits, 1), tf.int32) 
        num=tf.cast(tf.equal(reshape_logits, labels), dtype=tf.float32)
        # tf.nn.in_top_k(predictions, targets, k, name=None)
        # 每个样本的预测结果的前k个最大的数里面是否包括包含targets预测中的标签，一般取1，
        # 即取预测最大概率的索引与标签的对比
        top_k_op = tf.nn.in_top_k(t_logits, tf.reshape(labels,[-1]), 1)
        saver = tf.train.Saver()
        true_label = []
        pred_label = []
        # classes = ["BENIGN","Infiltration","DdoS","PortScan","Bot","FTP_Patator","SSH_Patator","Dos_slowloris","Dos_Slowhttptest","DoS_Hulk","Dos_GoldenEye","Heartbleed","Web_Attack_Brute_Force","Web_Attack_XSS","Web_Attack_Sql_Injection"]
        classes = ["BENIGN","ATTACK"]
        # classes = ["DdoS","PortScan"]
        with tf.Session() as session:
            ckpt = tf.train.get_checkpoint_state('model_folder/saver')
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(session, ckpt.model_checkpoint_path)
                #global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=session, coord=coord)
            true_count = 0
            for step in range(num_Iteration):

                features_test, labels_test = Train_model.next_batch(step,flag,BATCH_SIZE)
                #features_test=tf.reshape(features_test, [-1, 7, 7,1])
               
                
                test_logit,predictions,reshape_logits_,num_=session.run([t_logits,top_k_op,reshape_logits,num],feed_dict={features:features_test,labels:labels_test,keep_prob:1.0})
                true_label.extend(labels_test)
                pred_label.extend(reshape_logits_)
                true_count = true_count + np.sum(predictions)
            sess = tf.InteractiveSession()
            op = tf.confusion_matrix(true_label, pred_label)
            print ("confusion matrix in tensorflow: ")
            confusion_matrix = sess.run(op)
            # print(confusion_matrix)
            sess.close()

            # 计算各个类别的正确率
            acc = model.accuracyofeachclass(confusion_matrix, true_label, num_classes = 2)
            # print("true_count",true_count)
            # print(num_epoches * BATCH_SIZE)
            precision = true_count / (num_Iteration * BATCH_SIZE)
            print('测试precision=', precision)
            print(acc)
            plt.figure()
            plt.plot(classes, acc)
            plt.xlabel('accuracy')
            plt.ylabel('classification')
            plt.title('Accuracy of each classification')
            plt.savefig('model_folder/results/loss.png')
            plt.show()
           # evaluation()
            coord.request_stop()
            coord.join(threads)
        return precision,acc